# Robot search

In this notebook we introduce the RL framework for target searching with robots. It is based on the agents and environments presented in `rl_opts.rl_framework`. Some features of the code are:

- we use `numba` to improve speed.
- we implement more efficient ways of updating the H and G matrix (contribution by Dr. Michele Caraglio).
- we consider as base case `num_agents = 1`. 
- we modify the old version of our agents to make them more realistic and more similar to the dynamics of real robots.

In [ ]:
#| default_exp robot_search

#| hide
# Libs

Input libraries

In [ ]:
#| export
import numpy as np
from numba.experimental import jitclass
from numba import float64, int64, bool_, prange, njit
import math
import random

from rl_opts.utils import isBetween_c_Vec_numba, rand_choice_nb
from rl_opts.rl_framework.numba.agents import Forager

NOPYTHON = True

In [ ]:
#| hide
#| exec: false
# for debugging
NOPYTHON = False

In [ ]:
# | hide
import matplotlib.pyplot as plt

# Helpers

## Crossing targets while walking along an arc

Functions to evaluate whether the robot crosses a target while reorienting itself (action turn). During reorientation, the robot walks along a circumference arc.
- rotation_arc: creates the points of the arc (how many points are created can be chosen).
- arc_in_target_numba: evaluates whether the arc points are within a target area.

In [ ]:
# |export
@njit
def rotation_arc(theta : float, # Angle in rad (from origin, counterclockwise) describing the current orientation of the robot.
                 dtheta : float, # Angle difference in rad. How much the robot rotates when it decides to turn.
                 left : bool, # True if robot decided to turn left. False implies that the agent turns right.
                 num_points_arc:int=50, # Number of points that form the arc that is going to be evaluated.
                 R:float=1, # Rotation radius, distance from robot's center to the side. The robot reorients itself by rotating along the wheels of one side.
                 x:float=0, # Horizontal component of the robot's current position.
                 y:float=0 # Vertical component of the robot's current position.
                ):
    """Creates the points of the circumference arc covered by the robot while it reorients itself (action "turn")."""
    
    if left:
        value = 1
        sum_angle = 0
    else:
        value = -1
        sum_angle = np.pi
    
    phi = np.pi / 2 - theta
    
    # Get origin of rotation
    origin_x = x - value * R * np.cos(phi)
    origin_y = y + value * R * np.sin(phi)
    
    # Generate points for the arc in rad: from start point to final point = start point + length of arc
    start_angle = sum_angle - phi
    angles = np.linspace(start_angle, start_angle + dtheta, num_points_arc)
    x_arc = origin_x + R * np.cos(angles) #x positions of all the arc points
    y_arc = origin_y + R * np.sin(angles) #y positions of all the arc points
    
    return x_arc, y_arc

In [ ]:
# |export
@njit
def arc_in_target_numba(target_coord : np.array, # Array (number targets, 2) with the coordinates of all targets.
                        target_radius : float, # Radius of each target.
                        x_arc: np.array, # Horizontal components of arc points.
                        y_arc: np.array, # Vertical components of arc points.
                       )->np.array: # 1D array of size (number targets) with 1 on the indices of the found targets.
    """
    Checks which targets have been found by the agent when it reorients itself.
    """

    # Reshape the arc arrays to make them broadcastable
    x_arc_broadcasted = x_arc[:, np.newaxis]  # Shape (num_points_arc, 1)
    y_arc_broadcasted = y_arc[:, np.newaxis]  # Shape (num_points_arc, 1)

    # Compute the squared distances from all arc points to all target centers
    distances_squared = (x_arc_broadcasted - target_coord[:, 0])**2 + (y_arc_broadcasted - target_coord[:, 1])**2

    # Check if these distances are within the target radius squared
    mask_in_target = distances_squared <= target_radius**2

    # Summing over the arc points axis (axis=0), find which targets are found
    mask_found_targets = np.sign(np.sum(mask_in_target, axis=0))
    
    return mask_found_targets.astype('bool')

In [ ]:
targ = np.random.rand(100, 2)* 100
target_r = 1
a_x, a_y = rotation_arc(np.pi/6, np.pi/3, True, num_points_arc=50, R=1, x=0, y=0)
compile_first = arc_in_target_numba(targ, target_r, a_x, a_y)

In [ ]:
%timeit arc_in_target_numba(targ, target_r, a_x, a_y)

12.4 μs ± 21.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


# Environments

## Single Robot

Environment where a simulated robot (RL agent) searches for targets. The robot cannot see the targets. It gets a signal from the environment in the form of a positive reward whenever its position is within a target area.

- States: the agent perceives the counter of steps that it has been walking straight, i.e. it decided to continue in the same direction.

- Actions: "continue" , "turn left" or "turn right".
    - For "continue": walks straight in the same direction (no reorientation) for a distance $v$ sampled from a normal distribution. $N(\alpha, \sigma_\alpha)$
    - For "turning" actions: normal distribution centered on some angle to the left or to the right. $N(\beta, \sigma_\beta)$
    - When it turns, it just reorients (NO EXTRA DISPLACEMENT). The reorientation occurs with respect to a center that is on the side wheels (the ones that don't move), so (in contrast to our previous ideal agent) it also displaces its position when it reorients.

- Instead of kick, here we implement a time delay $\tau$. The agent needs to wait $\tau$ time steps from the moment it gets a target until it can get the same target again. 

- When the agent crosses a target, it resets the counter of steps walking straight. Its position doesn't change, it continues from where it was when it found the target.

In [ ]:
#|export
@jitclass([("target_positions", float64[:,:]) ,
           ("current_rewards", float64[:]) ,
           ("tau_allagents", float64[:]) ,
           ("depleted", float64[:,:]) ,
           ("current_directions", float64[:]) ,
           ("positions", float64[:,:]),
           ("previous_pos", float64[:,:]),
           ("mask", bool_[:])])
           
class RobotSearch():
    Nt : int
    L : float
    r : float
    tau : float
    agent_radius:float
    avg_vel:float
    std_vel:float
    avg_turn_angle:float
    std_turn_angle:float
    num_agents : int
    target_positions : np.ndarray
    current_rewards : np.array
    tau_allagents : np.array
    depleted : np.array
    current_directions : np.array
    positions : np.array
    previous_pos : np.array
    mask : np.array
    
    
    def __init__(self,
                 Nt:int, # Number of targets.
                 L:float, # Size of the (squared) world.
                 r:float, # Radius with center the target position. It defines the area in which agent detects the target.
                 tau:float, # Time it takes for the targets to replenish.
                 agent_radius:float, # Size of the agent (radius, from the center to the side).
                 avg_vel:float, # Mean displacement of the agent per time step (v). Normal distr.: N(v,std_v).
                 std_vel:float, # Std for the normal distr. of the agent's velocity (std_v).
                 avg_turn_angle:float, # Mean turning angle (in rad) when agent decides to turn (Theta). Normal distr.: N(Theta,std_theta).
                 std_turn_angle:float, # Std for the normal distr. of the agent's turning velocity (std_theta).
                 num_agents:int=1 # Number of agents that forage at the same time.
                 ):
        
        """        
        Class defining the foraging environment with replenishable targets modeled with a time delay instead of a kick. 
        It includes the methods to simulate the robot dynamics.
        
        """
        
        
        self.Nt = Nt
        self.L = L
        self.r = r
        self.agent_radius = agent_radius
        self.avg_vel = avg_vel
        self.std_vel = std_vel
        self.avg_turn_angle = avg_turn_angle
        self.std_turn_angle = std_turn_angle
        self.num_agents = num_agents
        
        # same replenishing time for all agents
        self.tau_allagents = tau * np.ones(self.num_agents)
        

        self.init_env()
        
    def init_env(self):
        """
        Environment initialization.
        """
        self.target_positions = np.random.rand(self.Nt, 2)*self.L
        
        #store who is/was rewarded
        self.current_rewards = np.zeros(self.num_agents)
        
        #number of steps since last target encounter, for each agent.
        self.depleted = np.zeros((self.Nt, self.num_agents))
        
        #set positions and directions of the agents
        self.current_directions = np.random.rand(self.num_agents)*2*np.pi
        self.positions = np.random.rand(self.num_agents, 2)*self.L
        self.previous_pos = self.positions.copy()       

        

    def update_pos(self, action, agent_index = 0):        
        """
        Updates information of the agent depending on its decision.

        Parameters
        ----------
        action : int
            0 (continue), 1 (left), 2 (right).
        agent_index : int, optional
            Index of the given agent. The default is 0.
            
        Returns
        ----------
        (x_arc, y_arc): (np.array, np.array)
            Positions of the turning arc. In case the agent does not turn, it returns (None, None).
        
        """
        
        # Save previous position to check if crossing happened
        self.previous_pos[agent_index] = self.positions[agent_index].copy()
        
        if action == 0: #continue
            # Update the velocity of the agent
            v = np.random.normal( max(0,self.avg_vel), max(0,self.std_vel))
            # Get the positional changes
            dx = v * np.cos(self.current_directions[agent_index])
            dy = v * np.sin(self.current_directions[agent_index])
            
            x_arc = None
            y_arc = None
            
        else:
            
            if action == 1: #left turn
                # Get angle difference
                dtheta = np.random.normal(self.avg_turn_angle, self.std_turn_angle)
                left = True

            elif action == 2: #right turn
                # Get angle difference
                dtheta = - np.random.normal(self.avg_turn_angle, self.std_turn_angle)
                left = False

            # Get new orientation of the agent
            new_theta = self.current_directions[agent_index] + dtheta
            # Get the positional changes
            dx =  np.sign(dtheta) * self.agent_radius * (np.sin(new_theta) - np.sin(self.current_directions[agent_index]))
            dy = - np.sign(dtheta) * self.agent_radius * (np.cos(new_theta) - np.cos(self.current_directions[agent_index]))
            
            # Get arc coordinates (default: 50 points)
            x_arc, y_arc = rotation_arc(self.current_directions[agent_index], dtheta, left, R=self.agent_radius, x=self.positions[agent_index][0], y=self.positions[agent_index][1])
            
            # Update agent's orientation
            self.current_directions[agent_index] = new_theta
        
       
        # Update the position
        self.positions[agent_index][0] += dx
        self.positions[agent_index][1] += dy
        
        return x_arc, y_arc
        
       
    def check_encounter(self, action, x_arc, y_arc, agent_index = 0):
        """
        Checks whether the agent found a target, and updates the information accordingly.

        Parameters
        ----------
        action : int
            Action index. Continue (0), turn left (1) or turn right (2).
        x_arc : np.array or None
            Horizontal components of the turning arc coordinates. None if agent walked straight.
        y_arc : np.array or None
            Vertical components of the turning arc coordinates. None if agent walked straight.
        agent_index : int, optional

        Returns
        -------
        True if the agent found a target.

        """
        
        # Check if there was a target encounter
        if action == 0:
            encounters = isBetween_c_Vec_numba(self.previous_pos[agent_index], self.positions[agent_index], self.target_positions, self.r)
        else:
            encounters = arc_in_target_numba(self.target_positions, self.r, x_arc, y_arc)
            
        #In case there were encounters:
        if np.sum(encounters) > 0: 
            # If there is more than 1 encounter, pick the closest to the agent.
            if sum(encounters) == 1:
                first_encounter = np.argwhere(encounters == True).flatten()
                
            else:
                # compute the distance from the previous position to each target            
                distance_previous_pos = np.sqrt((self.previous_pos[agent_index][0]- self.target_positions[:, 0])**2 + (self.previous_pos[agent_index][1] - self.target_positions[:, 1])**2)            
                # checking which encountered point is closer to previous position
                min_distance_masked = np.argmin(distance_previous_pos[encounters])
                first_encounter = np.argwhere(encounters == True)[min_distance_masked].flatten()
           
            # If the agent is NOT waiting for the recently found target to replenish:
            if not self.depleted[first_encounter, agent_index]:
                # We add the information that this agent got a reward
                self.current_rewards[agent_index] = 1
                # The replenishing time for this target and this agent starts.
                self.depleted[first_encounter, agent_index] += 1
                
                # The env issues a positive reward.
                return 1
            else:
                self.current_rewards[agent_index] = 0
                return 0
                    
        else:
            self.current_rewards[agent_index] = 0
            return 0   
    
    def update_replenishing_times(self, agent_index = 0):
        """
        Updates the replenishing times of the targets for a given agent.
        """
        self.depleted[np.argwhere(self.depleted[:, agent_index] != 0).flatten(), agent_index] += 1
        self.depleted[:, agent_index] = self.depleted[:, agent_index] % (self.tau_allagents[agent_index] + 1)
        
    def check_bc(self, agent_index = 0):
        """
        Updates position coordinates of a given agent to fulfill fixed boundary conditions.

        """
        self.positions[agent_index][0]  = max( 0, min( self.positions[agent_index][0] , self.L ) )
        self.positions[agent_index][1]  = max( 0, min( self.positions[agent_index][1] , self.L ) )

In [ ]:
#| hide

### Tests 

env = RobotSearch(Nt=100, 
                 L=100, 
                 r=1, 
                 tau=3, 
                 agent_radius=2, # Size of the agent (radius, from the center to the side).
                 avg_vel=1, # Mean displacement of the agent per time step (v). Normal distr.: N(v,std_v).
                 std_vel=0.3, # Std for the normal distr. of the agent's velocity (std_v).
                 avg_turn_angle=np.pi/6, # Mean turning angle (in rad) when agent decides to turn (Theta). Normal distr.: N(Theta,std_theta).
                 std_turn_angle=np.pi/18) # Std for the normal distr. of the agent's turning velocity (std_theta).

env.check_encounter(0,np.random.rand(20),np.random.rand(20))

# Learning

## Training loop

In [ ]:
#| export                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
@njit
def train_loop_robot(episodes : int, # Number of episodes to train
                   time_ep : int, # Length of episode
                   agent : object, # Agent class
                   env : object, # Environment class
                   h_mat_allT : bool = False # If True, returns the h_matrix at all times
                  )-> tuple: # Rewards and h-matrix of the trained agent      
    '''    
    Given an agent and environment, performs a loop train for the `RobotSearch` environment.
    '''

    if h_mat_allT: policy_t = np.zeros((episodes, agent.h_matrix.shape[-1]))
    
    save_rewards = np.zeros((episodes, time_ep))
    
    for ep in range(episodes):
        #initialize environment and agent's counter and g matrix
        env.init_env()
        agent.agent_state = 0
        agent.reset_g()

        for t in range(time_ep):
            agent.N_upd_H += 1
            agent.N_upd_G += 1
            
            #step to set counter to its min value n=1
            if t == 0 or env.current_rewards[0] == 1:
                #do one step with random direction (no learning in this step)
                action = rand_choice_nb(arr = np.arange(1, agent.num_actions), prob = np.array([1/len(np.arange(1,agent.num_actions))]*len(np.arange(1,agent.num_actions))))
                _,_ = env.update_pos(action)
                #check boundary conditions
                env.check_bc()
                #reset counter
                agent.agent_state = 0
                #update current rewards
                env.current_rewards[0] = 0
                reward = 0
                #update replenishing times
                env.update_replenishing_times()

            else:
                #get perception
                state = agent.get_state()
                #decide
                action = agent.deliberate(state)
                #act (update counter)
                agent.act(action)

                #update positions and get coordinates for turning arc (in case agent turned, otherwise these are None)
                x_arc, y_arc = env.update_pos(action)
                
                #update replenishing times
                env.update_replenishing_times()
                
                #check if target was found 
                reward = env.check_encounter(action, x_arc, y_arc)
                
                #check boundary conditions
                env.check_bc()
                #learn
                if reward == 1:
                    agent._learn_post_reward(reward)

            if agent.N_upd_H == agent.max_no_H_update-1:
                agent._learn_post_reward(reward)

            # Saving
            save_rewards[ep, t] = reward
            
        #Update of the H matrix at the end of the episode in case it was not updated in the last time steps
        if agent.N_upd_H != 0:
            agent._learn_post_reward(0)
        if h_mat_allT: policy_t[ep] = agent.h_matrix[0,:] / agent.h_matrix.sum(0)
        
      
    return (save_rewards, policy_t) if h_mat_allT else (save_rewards, agent.h_matrix)

In [ ]:
from rl_opts.rl_framework.numba.agents import Forager
from rl_opts.robot_search import RobotSearch, train_loop_robot  

In [ ]:
time_ep = 20000

env = RobotSearch(Nt=100, 
                 L=100, 
                 r=1, 
                 tau=3, 
                 agent_radius=2, # Size of the agent (radius, from the center to the side).
                 avg_vel=1, # Mean displacement of the agent per time step (v). Normal distr.: N(v,std_v).
                 std_vel=0.3, # Std for the normal distr. of the agent's velocity (std_v).
                 avg_turn_angle=np.pi/6, # Mean turning angle (in rad) when agent decides to turn (Theta). Normal distr.: N(Theta,std_theta).
                 std_turn_angle=np.pi/18) # Std for the normal distr. of the agent's turning velocity (std_theta).

agent = Forager(num_actions = 3, # From here are props of the agent (see Forager for details)
                gamma_damping = 0.00001,
                size_state_space = np.array([time_ep], dtype=np.int64),
                initial_prob_distr = (np.array([0.988, 0.012/2, 0.012/2])*np.ones((3, time_ep)).transpose()).transpose())


In [ ]:
rew, h = train_loop_robot(episodes = 50, time_ep = time_ep, agent = agent, env = env)

In [ ]:
import tracemalloc

# Start tracing memory allocations
tracemalloc.start()

# Run your function
rew, h = train_loop_robot(episodes = 50, time_ep = time_ep, agent = agent, env = env)

# Get the current memory usage and peak memory usage
current, peak = tracemalloc.get_traced_memory()

# Convert bytes to more readable units (e.g., MB)
print(f"Current memory usage: {current / 10**6:.2f} MB")
print(f"Peak memory usage: {peak / 10**6:.2f} MB")

# Stop tracing
tracemalloc.stop()

Current memory usage: 9.59 MB
Peak memory usage: 10.40 MB


## Parallel training

In [ ]:
#| export
@njit(parallel = True)
def run_robot_training_parallel(episodes, # Number of episodes
                                time_ep, # Length of episode
                                N_agents, # Number of agents               
                                h_mat_allT = False, # If to save the h_matrix at all times
                                Nt = 100, # From here are props of the environment (see TargetEnv for details) 
                                L = 100, 
                                r = 0.5, 
                                tau = 3,
                                agent_radius = 2,
                                avg_vel=1,
                                std_vel=0.3,
                                avg_turn_angle=np.pi/6,
                                std_turn_angle=np.pi/18,
                                num_actions = 3, # From here are props of the agent (see Forager for details)
                                size_state_space = np.array([100], dtype=np.int64), 
                                gamma_damping = 0.00001,
                                eta_glow_damping = 0.1,
                                initial_prob_distr = np.array([[],[],[]]),
                                policy_type = 'standard', 
                                beta_softmax = 3,  
                                fixed_policy = np.array([[],[]]),
                                max_no_H_update = int(1e3)
                               ):
    
    save_rewards = np.zeros((N_agents, episodes))
    
    if h_mat_allT:
        save_h_matrix = np.zeros((N_agents, episodes, size_state_space[0]))  
    else:        
        save_h_matrix = np.zeros((N_agents, num_actions, size_state_space[0])) 
    
    for n_agent in prange(N_agents):
        
        env = RobotSearch(Nt, L, r, tau, agent_radius, avg_vel, std_vel, avg_turn_angle, std_turn_angle,1)
        
        agent = Forager(num_actions,size_state_space,gamma_damping,
                        eta_glow_damping,policy_type,beta_softmax,
                        initial_prob_distr,fixed_policy,max_no_H_update,'s')
                          
        rews, mat = train_loop_robot(episodes, time_ep, agent, env, h_mat_allT)

             
        #mean reward per episode
        for t in range(episodes):
            save_rewards[n_agent, t] = np.mean(rews[t])
        
        save_h_matrix[n_agent] = mat
        
    return save_rewards, save_h_matrix

In [ ]:
from rl_opts.robot_search import RobotSearch, train_loop_robot
from rl_opts.rl_framework.numba.agents import Forager

In [ ]:
EPISODES = 9
TIME_EP = 10
N_AGENTS = 4


rewards, h_matrix = run_robot_training_parallel(episodes = int(EPISODES), 
                                                time_ep = int(TIME_EP), 
                                                N_agents = N_AGENTS)

In [ ]:
rewards.shape

(4, 9)

# Walk from policy

These replicate what we were doing in `rl_opts.learn_and_bench.walk_from_policy` and help get efficiencies for fixed policies.

In [ ]:
#| export   
@njit
def single_robot_trajectory(time_ep : int, # Length of each run / episode
                            policy : np.array, # Policy of the walker (rows: states. columns:prob of doing each action)
                            env : object # Environment where the walker moves
                           )-> tuple: # number of visited targets, agent and target positions

    '''
    Walk of a single agent in env of type RobotSearch given a policy. Performance is evaluated as the number of targets found in a fixed time time_ep.
    '''
    
    number_visited_targets = 0
    agent_positions = np.zeros((time_ep, 2))
    rew_per_timestep = np.zeros(time_ep)
    
    #initialize environment and agent's counter and g matrix
    env.init_env()
    agent_state = 0

    for t in range(time_ep):
        
        agent_positions[t,:] = env.positions[0] #position of agent with index 0
        
        if t == 0 or env.current_rewards[0] == 1:
            # change direction
            action = rand_choice_nb(arr = np.arange(1, len(policy[0])), prob = np.array([1/len(np.arange(1,len(policy[0])))]*len(np.arange(1,len(policy[0])))))
            _,_ = env.update_pos(action)
            #check boundary conditions
            env.check_bc()
            #reset counter
            agent_state = 0
            #update current rewards
            env.current_rewards[0] = 0
            reward = 0
            #update replenishing times
            env.update_replenishing_times()
            

        else: 
            # decide       
            action = rand_choice_nb(arr = np.arange(len(policy[0])), prob = policy[agent_state])
            #update positions
            x_arc, y_arc = env.update_pos(action)
            #update replenishing times
            env.update_replenishing_times()
            
            #check if target was found 
            reward = env.check_encounter(action, x_arc, y_arc)
            
            #check boundary conditions
            env.check_bc()
            
            # update agent_state
            if action == 0: #continue
                agent_state += 1
            else: #turn
                agent_state = 0

            number_visited_targets += reward
            rew_per_timestep[t] += reward
                
    return (number_visited_targets, agent_positions, env.target_positions, rew_per_timestep) 

In [ ]:
#| export
@njit(parallel = True)
def parallel_robots_trajectories(time_ep : int, # Length of each run / episode
                                 N_agents : int, # Number of agents to consider
                                 Nt = 100, # Number of targets in the environment
                                 L = 100, # Size of the environment
                                 r = 0.5, # Radius of the targets
                                 tau = 3, # Time delay until target is available again
                                 agent_radius = 2,
                                 avg_vel=1,
                                 std_vel=0.3,
                                 avg_turn_angle=np.pi/6,
                                 std_turn_angle=np.pi/18,
                                 policies = np.array([[1.0,0.0,0.0,0.0], [0.0,1.0,0.0,0.0]]) # Policy of the agents
                                )-> tuple: # number of visited targets, agent and target positions
    """
    Runs in parallel single_robot_trajectory. Due to numba props, we need to give all parameters as inputs (see source).
    """
    
    agent_positions = np.zeros((N_agents, time_ep, 2))
    number_visited_targets = np.zeros(N_agents)
    target_positions = np.zeros((N_agents, Nt, 2))
    reward_per_timestep = np.zeros((N_agents, time_ep))
    
    for n_agent in prange(N_agents):
        
        env = RobotSearch(Nt, L, r, tau, agent_radius, avg_vel, std_vel, avg_turn_angle, std_turn_angle,1)
        
        num_tar, pos, tar_pos, rew_t = single_robot_trajectory(time_ep, policies[n_agent], env) 
    
        agent_positions[n_agent] = pos
        number_visited_targets[n_agent] = num_tar
        target_positions[n_agent] = tar_pos
        reward_per_timestep[n_agent] = rew_t
        
    return (number_visited_targets, agent_positions, target_positions, reward_per_timestep)

#| hide
# nbdev

In [ ]:
#| hide
from nbdev import nbdev_export ; nbdev_export()